In [4]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
df = pd.read_csv('C:/Users/egory/Desktop/Полезности/userbehaviour.csv')

1. Сплит пользователей на контроль и тест 

In [13]:
np.random.seed(42)

In [17]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [19]:
df['group'] = np.random.choice(['control', 'test'], size=len(df))

2. Определим метрики 

In [22]:
binary_metrics = ["Left Review"]
continuous_metrics = ["Average Spent on App (INR)", "Average Screen Time", "Ratings"]

3. Z-тест для бинарной метрики 

In [25]:
def two_prop_ztest(success_a, n_a, success_b, n_b):
    p1, p2 = success_a / n_a, success_b / n_b
    p = (success_a + success_b) / (n_a + n_b)   
    se = np.sqrt(p * (1 - p) * (1/n_a + 1/n_b))
    z = (p2 - p1) / se
    pval = 2 * (1 - stats.norm.cdf(abs(z)))     
    return p1, p2, z, pval

In [37]:
m = "Left Review" 
control = df[df["group"] == "control"][m]
test = df[df["group"] == "test"][m]

p1, p2, z, pval = two_prop_ztest(control.sum(), len(control), test.sum(), len(test))

print(f"Метрика: {m}")
print(f"Control mean = {p1:.3f}, Test mean = {p2:.3f}")
print(f"Z-stat = {z:.3f}, p-value = {pval:.4f}")

Метрика: Left Review
Control mean = 0.483, Test mean = 0.512
Z-stat = 0.921, p-value = 0.3570


Эксперимент в текущем виде не показал значимого эффекта: изменений в поведении пользователей нет (с точки зрения статистики)

4. t-тест для непрерывных метрик

In [45]:
for m in continuous_metrics:
    control = df[df["group"] == "control"][m]
    test = df[df["group"] == "test"][m]
    t_stat, pval = stats.ttest_ind(control, test, equal_var=False)
    print(f"\nМетрика: {m}")
    print(f"Control mean = {control.mean():.2f}, Test mean = {test.mean():.2f}")
    print(f"t-stat = {t_stat:.3f}, p-value = {pval:.4f}")


Метрика: Average Spent on App (INR)
Control mean = 416.47, Test mean = 432.03
t-stat = -0.787, p-value = 0.4315

Метрика: Average Screen Time
Control mean = 23.92, Test mean = 24.84
t-stat = -1.026, p-value = 0.3050

Метрика: Ratings
Control mean = 6.45, Test mean = 6.57
t-stat = -0.705, p-value = 0.4808


Все проверенные метрики (расходы, экранное время, рейтинги) показывают чуть лучшие результаты в тесте, но во всех случаях p-value значительно выше 0.05. Это значит, что эксперимент не выявил статистически значимых различий между тестовой и контрольной группами.